## <font color = 'skyblue'>PROYECTO SOBRE EL TRÁFICO AÉREO EN 2020 Y 2021

<font color = 'aquamarine'> Importación de todos los paquetes que vamos a necesitar para el análisis de los datos.

In [43]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import math 
import io

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns
import pylab as py
import sidetable

# Gráficos
# ==============================================================================
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Asunciones y Preprocesamiento
# ==============================================================================
from scipy import stats
import math
from scipy.stats import levene
from sklearn.preprocessing import StandardScaler
import itertools

# ANOVA
# ==============================================================================
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from statsmodels.tools.tools import add_constant
from statsmodels.stats.outliers_influence import variance_inflation_factor


#Linear Regresion con Sklearn
# ==============================================================================
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error


# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

### <font color = 'sky blue'>Este será el primer dataset con el que trabajaremos. Los datos de 2020.

In [3]:
aena_2020 = 'DEFINITIVOS_2020.xls'

df_aena_2020 = pd.read_excel(aena_2020)

In [4]:
df_aena_2020.head(20)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,Dirección de Operaciones,NaN,NaN,"TRÁFICO DE PASAJEROS, OPERACIONES Y CARGA EN L...",NaN,NaN,NaN,NaN,NaN,NaN,AÑO 2020
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LLEGADA Y SALIDA
2,NaN,NaN,NaN,NaN,NaN,NaN,DATOS DEFINITIVOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Departamento de Estadísticas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,PASAJEROS,NaN,NaN,NaN,OPERACIONES,NaN,NaN,NaN,MERCANCÍA,NaN
6,NaN,NaN,\nAEROPUERTOS,NaN,\nTotal,% Inc 2020 /s,NaN,\nAEROPUERTOS,\nTotal,% Inc 2020 /s,NaN,\nAEROPUERTOS,\nTotal,% Inc 2020 /s
7,NaN,NaN,NaN,NaN,NaN,2019,NaN,NaN,NaN,2019,NaN,NaN,NaN,2019
8,NaN,NaN,ADOLFO SUÁREZ MADRID-BARAJAS,NaN,17112246,-0.722811,NaN,ADOLFO SUÁREZ MADRID-BARAJAS,165743,-0.611274,NaN,ADOLFO SUÁREZ MADRID-BARAJAS,402575121,-0.281166
9,NaN,NaN,BARCELONA-EL PRAT J.T.,NaN,12738769,-0.758225,NaN,BARCELONA-EL PRAT J.T.,122638,-0.644077,NaN,ZARAGOZA,143377681,-0.214881


### <font color = 'sky blue'> Como hemos visto, el archivo original es un xls. Tenemos varias columnas vacías provenientes de la naturaleza del excel, junto con filas que en un dataset no nos aportan información. Limpiar esto será lo primero que haremos.

In [5]:
df_aena_2020 = df_aena_2020.iloc[8:]

In [6]:
df_aena_2020.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 3', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 10', 'Unnamed: 11'], axis=1, inplace=True)

<font color = 'sky blue'> Ahora vamos a renombrarlas con los nombres originales que hemos visto en el pdf adjunto para saber a qué hacen referencia esas columnas.

In [7]:
df_aena_2020.rename(columns={'Unnamed: 2': 'Aeropuertos', 'Unnamed: 4': 'Total_pasajeros', 'Unnamed: 5': '%_inc_2020_/s_2019_pasajeros', 
                             'Unnamed: 8': 'Total_operaciones', 'Unnamed: 9': '%_inc_2020_/s_2019_operaciones','Unnamed: 12': 'Total__carga_mercancía', 
                             'Unnamed: 13': '%_inc_2020_/s_2019_mercancia'}, inplace=True)

In [8]:
df_aena_2020.reset_index(drop=True, inplace=True)

In [9]:
df_aena_2020.head()

,Aeropuertos,Total_pasajeros,%_inc_2020_/s_2019_pasajeros,Total_operaciones,%_inc_2020_/s_2019_operaciones,Total__carga_mercancía,%_inc_2020_/s_2019_mercancia
0,ADOLFO SUÁREZ MADRID-BARAJAS,17112246,-0.722811,165743,-0.611274,402575121,-0.281166
1,BARCELONA-EL PRAT J.T.,12738769,-0.758225,122638,-0.644077,143377681,-0.214881
2,PALMA DE MALLORCA,6108514,-0.794472,76852,-0.646205,114449796,-0.352652
3,MALAGA-COSTA DEL SOL,5161243,-0.740101,67280,-0.467936,64334516,-0.002112
4,GRAN CANARIA,5134252,-0.612837,59670,-0.58831,13926469,-0.294068


In [44]:
io.open('DEFINITIVOS_2021.xls', encoding='latin-1')

<_io.TextIOWrapper name='DEFINITIVOS_2021.xls' mode='r' encoding='latin-1'>

In [42]:
df_aena_2021 = pd.read_csv("DEFINITIVOS_2021_2003.txt", sep= ";")
df_aena_2021.head(2)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 31: invalid continuation byte

In [41]:
df_aena_2021.shape

(4, 1)